In [ ]:
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import gymnasium as gym
from source.algorithms import *
import source.config as config
import source.farm_env # Import necessary to register the gym environment

In [ ]:
import numpy as np
import math

import matplotlib.pyplot as plt

rewards = [-0.999, -0.989, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, -0.8, 0.14, 0.145, 0.17, 0.185, 0.21, 0.205, -0.7, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, -0.72, 0.05]

gaussian_rewards = []
sigma = 6

for i, reward in enumerate(rewards):
    reward = np.clip(reward, -0.999, 0.999)
    std_reward = (reward - (-0.999)) / (0.999 - (-0.999))
    scaled_reward = std_reward * (1 - -1) + -1
    gaussian_modifier = math.exp(-((i+1) - 30)**2 / (2 * (sigma**2)))
    gaussian_rewards.append(scaled_reward * gaussian_modifier)

print(gaussian_rewards)

plt.figure(figsize=(12, 8))
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.plot(range(len(gaussian_rewards)), gaussian_rewards, color='#acc700')
plt.xlabel('Year', size=20)
plt.ylabel('Gaussian Reward', size=20)

# Add red text for negative rewards
for i, reward in enumerate(rewards):
    d, r, c = 1, 1, 0
    if reward < 0:
        if i == 1:
            d, r = -0.7, 1.5
        if i == 0:
            c = 0.2
        plt.text(i*r+c, gaussian_rewards[i] - 0.04 * d, "Buy a sheep", color='#ee5c59', size=10, ha='center')
        plt.plot(i, gaussian_rewards[i], 'o', color='#ee5c59')  # Add red dot

plt.show()


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

def gaussian(x, c, sigma):
    return np.exp(-((x - c) / (np.sqrt(2) * sigma)) ** 2)

def neg_gaussian(x, c, sigma):
    return -gaussian(x, c, sigma)

c = 30
sigma = 6

x = np.linspace(0, 60, 1000)
y_positive = gaussian(x, c, sigma)
y_negative = neg_gaussian(x, c, sigma)

plt.figure(figsize=(8, 4))
plt.plot(x, y_positive, color='#acc700', linewidth=2, label='Positive $\Delta$')
plt.plot(x, y_negative, color='#ee5c59', linewidth=2, label='Negative $\Delta$')
plt.title(f'Gaussian transformation ($\mu$={c}, $\sigma$={sigma})')
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend()
plt.show()

Correlation final sheep_count/budget

In [ ]:
import source.config as config
from source.algorithms import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import source.farm_env # Import necessary to register the gym environment

In [ ]:
env = gym.make(id='FarmEnv-v0',
                initial_budget = config.INITIAL_BUDGET,
                sheep_cost = config.SHEEP_COST,
                wheat_cost = config.WHEAT_COST,
                wool_price = config.WOOL_PRICE,
                wheat_price = config.WHEAT_PRICE,
                max_years = config.MAX_YEARS,
                wool_fixed_cost = config.WOOL_FIXED_COST,
                storm_probability = config.STORM_PROBABILITY,
                incest_penalty = config.INCEST_PENALTY,
                reward_std = config.SIGMA
                )

In [ ]:
learning_rate = 0.001
policy_learning_rate = 1e-4
value_learning_rate = 1e-4
n_episodes = 300_000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes)  # reduce the exploration over time
final_epsilon = 0.1

REINFORCE_agent = FarmAgentNeuralREINFORCEAdvantage(
    environment=env, policy_learning_rate=policy_learning_rate, value_learning_rate=value_learning_rate, epsilon=start_epsilon, epsilon_decay=epsilon_decay, final_epsilon=final_epsilon, gamma=.999,
    policy_net_weights_path=r'agent_models\REINFORCENeuralAdvantage\s6-penalty2.5-edecay1-g0.999\policy_net_weights-500k.pth',
    value_net_weights_path=r'agent_models\REINFORCENeuralAdvantage\s6-penalty2.5-edecay1-g0.999\value_net_weights-500k.pth'
)

In [ ]:
def generate_trajectories(env, REINFORCE_agent, num_trajectories=50000, trajectory_length=30):
    final_sheep_counts = []
    final_budgets = []

    for _ in tqdm(range(num_trajectories)):
        state, info = env.reset()
        for _ in range(trajectory_length):
            action = REINFORCE_agent.policy(state, greedy=True)
            s_prime, reward, terminated, truncated, info = env.step(action)
            
            if terminated or truncated:
                break
            else:
                state = s_prime

        final_sheep_counts.append(state['sheep_count'])
        final_budgets.append(state['budget'])

    return final_sheep_counts, final_budgets

In [ ]:
# Generate trajectories
final_sheep_counts, final_budgets = generate_trajectories(env, REINFORCE_agent)

In [ ]:
# Create scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(final_sheep_counts, final_budgets, alpha=0.1, color='#acc700')
plt.xlabel('Final Sheep Count', size=20)
plt.ylabel('Final Budget', size=20)
# plt.title('Correlation between Final Sheep Count and Final Budget')


# Add trend line
z = np.polyfit(final_sheep_counts, final_budgets, 1)
p = np.poly1d(z)
plt.plot(final_sheep_counts, p(final_sheep_counts), "--", alpha=0.8, color='#ee5c59')
plt.xlim(0, 100)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

################################################
# CORRELATION
# Calculate and display correlation coefficient
correlation = np.corrcoef(final_sheep_counts, final_budgets)[0, 1]
plt.text(0.05, 0.85, f'Correlation: {correlation:.2f}', transform=plt.gca().transAxes, size=30)

plt.tight_layout()
plt.savefig('media/REINFORCENeuralAdvantage/correlation_sheep_budget-500k-s612333345654321fbcvbzbbxc.png', dpi=300, bbox_inches='tight')
plt.show()

# Print summary statistics
print(f"Average final sheep count: {np.mean(final_sheep_counts):.2f}")
print(f"Average final budget: {np.mean(final_budgets):.2f}")
print(f"Correlation coefficient: {correlation:.2f}")